## Import dependencies

In [45]:
import requests
import pandas as pd 
import os 
import transform_functions as tf
from credentials import *  

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Get Australian Capital Cities
Read in CSV of australian capital cities

In [46]:
capital_cities_df = pd.read_csv(os.path.join("..", "data", "australian_capital_cities.csv"))

## Get Weather Data 
Get weather data by requesting from openweathermap REST APIs for each capital city 

In [47]:
weather_data = []
for city_name in capital_cities_df["city_name"]:
    params = {
        "q": city_name,
        "appid": api_key
    }   
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather", params=params)
    if response.status_code == 200: 
        weather_data.append(requests.get(f"http://api.openweathermap.org/data/2.5/weather", params=params).json())
    else: 
        raise Exception("Extracting weather api data failed. Please check if API limits have been reached.")

## Read JSON data into Pandas DataFrame

In [48]:
weather_df = pd.json_normalize(weather_data)
weather_df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,wind.gust
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1638370219,39600,2172517,Canberra,200,149.1281,-35.2835,...,82,2.06,90,2,2,2004200,AU,1638384133,1638435824,NaN
1,"[{'id': 801, 'main': 'Clouds', 'description': ...",stations,10000,1638370479,39600,2147714,Sydney,200,151.2073,-33.8679,...,90,1.34,340,20,2,2001174,AU,1638383850,1638435109,1.34
2,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1638370203,34200,2073124,Darwin,200,130.8418,-12.4611,...,83,1.54,290,0,1,9574,AU,1638391323,1638437414,NaN
3,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1638370381,36000,2174003,Brisbane,200,153.0281,-27.4679,...,86,0.45,175,75,2,2005393,AU,1638384296,1638433788,2.68
4,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1638369883,37800,2078025,Adelaide,200,138.6000,-34.9333,...,72,1.54,160,0,2,2001763,AU,1638386713,1638438299,NaN


## Convert unix timestamp to datetime timestamp string 

In [49]:
date_fixed_weather_df = tf.convert_unix_timestamp(input_df = weather_df, date_columns=["dt"])
date_fixed_weather_df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,wind.gust
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:19,39600,2172517,Canberra,200,149.1281,-35.2835,...,82,2.06,90,2,2,2004200,AU,1638384133,1638435824,NaN
1,"[{'id': 801, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:54:39,39600,2147714,Sydney,200,151.2073,-33.8679,...,90,1.34,340,20,2,2001174,AU,1638383850,1638435109,1.34
2,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:03,34200,2073124,Darwin,200,130.8418,-12.4611,...,83,1.54,290,0,1,9574,AU,1638391323,1638437414,NaN
3,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:53:01,36000,2174003,Brisbane,200,153.0281,-27.4679,...,86,0.45,175,75,2,2005393,AU,1638384296,1638433788,2.68
4,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:44:43,37800,2078025,Adelaide,200,138.6000,-34.9333,...,72,1.54,160,0,2,2001763,AU,1638386713,1638438299,NaN


## Replace column names

In [50]:
clean_weather_df = tf.replace_column_character(date_fixed_weather_df, {".": "_"})
clean_weather_df.head()

,weather,base,visibility,dt,timezone,id,name,cod,coord_lon,coord_lat,...,main_humidity,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,wind_gust
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:19,39600,2172517,Canberra,200,149.1281,-35.2835,...,82,2.06,90,2,2,2004200,AU,1638384133,1638435824,NaN
1,"[{'id': 801, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:54:39,39600,2147714,Sydney,200,151.2073,-33.8679,...,90,1.34,340,20,2,2001174,AU,1638383850,1638435109,1.34
2,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:03,34200,2073124,Darwin,200,130.8418,-12.4611,...,83,1.54,290,0,1,9574,AU,1638391323,1638437414,NaN
3,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:53:01,36000,2174003,Brisbane,200,153.0281,-27.4679,...,86,0.45,175,75,2,2005393,AU,1638384296,1638433788,2.68
4,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:44:43,37800,2078025,Adelaide,200,138.6000,-34.9333,...,72,1.54,160,0,2,2001763,AU,1638386713,1638438299,NaN


## Rename fields

In [51]:
clean_weather_df = clean_weather_df.rename(columns={
    "id":"city_id", 
    "dt": "datetime"
})
clean_weather_df.head()

,weather,base,visibility,datetime,timezone,city_id,name,cod,coord_lon,coord_lat,...,main_humidity,wind_speed,wind_deg,clouds_all,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,wind_gust
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:19,39600,2172517,Canberra,200,149.1281,-35.2835,...,82,2.06,90,2,2,2004200,AU,1638384133,1638435824,NaN
1,"[{'id': 801, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:54:39,39600,2147714,Sydney,200,151.2073,-33.8679,...,90,1.34,340,20,2,2001174,AU,1638383850,1638435109,1.34
2,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:50:03,34200,2073124,Darwin,200,130.8418,-12.4611,...,83,1.54,290,0,1,9574,AU,1638391323,1638437414,NaN
3,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,2021-12-01 14:53:01,36000,2174003,Brisbane,200,153.0281,-27.4679,...,86,0.45,175,75,2,2005393,AU,1638384296,1638433788,2.68
4,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,2021-12-01 14:44:43,37800,2078025,Adelaide,200,138.6000,-34.9333,...,72,1.54,160,0,2,2001763,AU,1638386713,1638438299,NaN


## Create City DataFrame

In [52]:
city_df = clean_weather_df[["city_id", "name", "coord_lon", "coord_lat"]].drop_duplicates() 
city_df.head()

,city_id,name,coord_lon,coord_lat
0,2172517,Canberra,149.1281,-35.2835
1,2147714,Sydney,151.2073,-33.8679
2,2073124,Darwin,130.8418,-12.4611
3,2174003,Brisbane,153.0281,-27.4679
4,2078025,Adelaide,138.6000,-34.9333


## Create Temperature DataFrame

In [53]:
temperature_df = clean_weather_df[["city_id", "datetime", "main_temp", "main_feels_like", "main_temp_min", "main_temp_max"]]
temperature_df.head()

,city_id,datetime,main_temp,main_feels_like,main_temp_min,main_temp_max
0,2172517,2021-12-01 14:50:19,288.08,287.77,286.51,289.12
1,2147714,2021-12-01 14:54:39,291.28,291.50,289.76,292.76
2,2073124,2021-12-01 14:50:03,301.03,305.40,300.87,302.53
3,2174003,2021-12-01 14:53:01,294.70,295.16,293.87,295.45
4,2078025,2021-12-01 14:44:43,291.27,291.02,288.07,292.83


## Create Atmosphere DataFrame

In [54]:
atmosphere_df = clean_weather_df[["city_id", "datetime", 'main_pressure', 'main_humidity']]
atmosphere_df.head()

,city_id,datetime,main_pressure,main_humidity
0,2172517,2021-12-01 14:50:19,1016,82
1,2147714,2021-12-01 14:54:39,1016,90
2,2073124,2021-12-01 14:50:03,1008,83
3,2174003,2021-12-01 14:53:01,1012,86
4,2078025,2021-12-01 14:44:43,1012,72


## Create SQL Connection

In [55]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from urllib.parse import quote_plus as urlquote
connection_url = URL.create(
    drivername = "postgresql", 
    username = db_user,
    password = db_password,
    host = "localhost", 
    port = 5432,
    database = "weather_db", 
)

engine = create_engine(connection_url)

## Reflect ORM

In [56]:
from sqlalchemy import MetaData
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine)
city = metadata_obj.tables["city"]
temperature = metadata_obj.tables["temperature"]
atmosphere = metadata_obj.tables["atmosphere"]


## Upsert: City

In [57]:
insert_statement = postgresql.insert(city).values(city_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['city_id'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['city_id']})
engine.execute(upsert_statement)

## Upsert: Temperature

In [58]:
insert_statement = postgresql.insert(temperature).values(temperature_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['city_id', 'datetime'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['city_id', 'datetime']})
engine.execute(upsert_statement)

## Upsert Atmosphere

In [59]:
insert_statement = postgresql.insert(atmosphere).values(atmosphere_df.to_dict(orient='records'))
upsert_statement = insert_statement.on_conflict_do_update(
    index_elements=['city_id', 'datetime'],
    set_={c.key: c for c in insert_statement.excluded if c.key not in ['city_id', 'datetime']})
engine.execute(upsert_statement)

In [1]:
import datetime as dt 
print(f"ETL job completed at {dt.datetime.now()}")

ETL job completed at 2021-12-01 23:30:01.961751
